In [1]:
import os
import numpy as np
import pandas as pd
from faker import Faker
from datetime import date

In [2]:
fake = Faker('es_MX')

if not os.path.exists('../datasets/'):
    print('Creating datasets directory')
    os.mkdir('../datasets/')

In [3]:
def rand_date():
    year = np.random.randint(2000, 2018)
    month = np.random.randint(1, 12)
    day = np.random.randint(1, 31)

    while month == 2 and day > 28:
        day = np.random.randint(1, 28)

    return date(year, month, day)

def to_roman(num):
    conv = [
        [1000, 'M'], [900, 'CM'], [500, 'D'], [400, 'CD'],
        [ 100, 'C'], [ 90, 'XC'], [ 50, 'L'], [ 40, 'XL'],
        [  10, 'X'], [  9, 'IX'], [  5, 'V'], [  4, 'IV'],
        [   1, 'I']
    ]
    roman = ''
    i = 0
    
    while num > 0:
        while conv[i][0] > num: 
            i+=1
        roman += conv[i][1]
        num -= conv[i][0]
    
    return roman

## Dimensiones

In [4]:
class Cliente():
    def __init__(self, **kwargs):
        self.fecha_creacion = str(kwargs['c_date'])
        self.fecha_ultima_compra = str(kwargs['uc_date'])
        self.fecha_ultima_devolucion = str(kwargs['dev_date'])
        self.total_compras = kwargs['t_compras']
        self.cantidad_devoluciones = kwargs['cant_dev']
    
    def __str__(self):
        return str(self.__dict__)
    
    def to_dict(self):
        return self.__dict__

class Rating(object):
    def __init__(self, stars, comments):
        self.cant_estrellas = stars
        self.comentarios = comments
        
    def __str__(self):
        return str(self.__dict__)
    
    def to_dict(self):
        return self.__dict__

class Empleado(object):
    def __init__(self, **kwargs):
        self.nombre = kwargs['name']
        self.cargo = kwargs['charge']
        self.sueldo = kwargs['salary']
        self.fecha_ingreso = str(kwargs['c_date'])
        self.fecha_desvinculacion = str(kwargs['c_desvin'])
        self.horas_trabajadas = str(kwargs['w_hours'])
    
    def __str__(self):
        return str(self.__dict__)
    
    def to_dict(self):
        return self.__dict__

class Producto(object):
    def __init__(self, name, category, price):
        self.nombre = name
        self.categoria = category
        self.precio = price
    
    def __str__(self):
        return str(self.__dict__)
    
    def to_dict(self):
        return self.__dict__

class Sucursal(object):
    def __init__(self, _dir, region, time_off):
        self.direccion = _dir
        self.region = region
        self.tiempo_offline_horas = time_off
    
    def __str__(self):
        return str(self.__dict__)
    
    def to_dict(self):
        return self.__dict__

class Costos(object):
    def __init__(self, **kwargs):
        self.gastos_admin = kwargs['g_admin']
        self.gastos_no_operacionales = kwargs['gn_oper']
        self.arriendos = kwargs['arr']
        self.fecha_mensual = str(kwargs['f_month'])
    
    def __str__(self):
        return str(self.__dict__)
    
    def to_dict(self):
        return self.__dict__

## Funciones generadoras

In [5]:
def get_cliente():
    c_date = rand_date()
    uc_date = rand_date()
    dev_date = rand_date()
    
    while uc_date < c_date:
        uc_date = rand_date()
    
    while uc_date < dev_date:
        dev_date = rand_date()
        
    t_compras = np.random.randint(1, 500)
    cant_dev = np.random.randint(0, 50)
    
    return Cliente(**{
        'c_date': c_date, 
        'uc_date': uc_date,
        'dev_date': dev_date,
        't_compras': t_compras,
        'cant_dev': cant_dev,
    })

def get_rating():
    return Rating(np.random.randint(0, 10), fake.text())

def get_empleado():
    nombre = fake.name()
    cargos = ['Vendedor', 'Cajero', 'Gerente', 'Auxiliar', 'Seguridad', 'Reponedor']
    sueldos = [600000, 500000, 800000, 500000, 550000, 400000]
    std = np.ndarray.std(np.array(sueldos))
    std = np.random.choice([std, -std, std*.25, -std*.25, std*.50, -std*.50, std*.75, -std*.75])
    cargo = np.random.choice(cargos)
    idx_cargo = cargos.index(cargo)
    sueldo = int(sueldos[idx_cargo] + std)
    c_date = rand_date()
    c_desvin = rand_date()
    
    while c_date > c_desvin:
        c_desvin = rand_date()
    
    w_hours = (date.today() - c_date).days * 9 * np.random.choice([1.0, 0.95, 0.90, 0.85, 0.80])
    w_hours = int(w_hours)
    
    return Empleado(**{
        'name': nombre,
        'charge': cargo,
        'salary': sueldo,
        'c_date': c_date,
        'c_desvin': c_desvin,
        'w_hours': w_hours
    })

def get_producto():
    name = fake.text().split(' ')[0]
    cat = ['Camiseta', 'Jersey', 'Chaqueta', 'Abrigo', 'Vaqueros', 'Calcetines', 'Pantalon Corto', 'Chandal']
    cat = np.random.choice(np.array(cat))
    price = np.random.randint(20, 100) * 1000
    
    return Producto(name, cat, price)

def get_sucursal(reg):
    _dir = ' '.join(fake.text().split(' ')[:3])
    reg = to_roman(reg)
    reg = reg + ' Region'
    offline = np.random.randint(1, 100)
    
    return Sucursal(_dir, reg, offline)

def get_costos(year, month):
    g_admin = np.random.randint(1000000, 10000000)
    g_nop = np.random.randint(1000000, 10000000)
    arr = np.random.randint(10000000, 100000000)
    day = np.random.randint(1, 31)
    
    while month == 2 and day > 28:
        day = np.random.randint(1, 28)
    
    f_month = date(year, month, day)
    
    return Costos(**{
        'g_admin': g_admin,
        'gn_oper': g_nop,
        'arr': arr,
        'f_month': f_month
    })

## Generar Datos

In [6]:
sucursales = []
costos = []
productos = []
empleados = []
ratings = []
clientes = []

In [7]:
# 2 sucursales por region (5, 8 y RM), las online cuentan como sucursal aparte en la misma region

suc = get_sucursal(5)
suc.tiempo_offline_horas = 0
sucursales.append(suc)

suc = get_sucursal(8)
suc.tiempo_offline_horas = 0
sucursales.append(suc)

suc = get_sucursal(0)
suc.region = 'Region Metropolitana'
suc.tiempo_offline_horas = 0
sucursales.append(suc)

suc = get_sucursal(5)
suc.direccion = 'https://tallawrande.cl'
sucursales.append(suc)

suc = get_sucursal(8)
suc.direccion = 'https://tallawrande.cl'
sucursales.append(suc)

suc = get_sucursal(0)
suc.region = 'Region Metropolitana'
suc.direccion = 'https://tallawrande.cl'
sucursales.append(suc)


print(len(sucursales), 'sucursales creadas')
print('Primera sucursal de la lista:', sucursales[0])

6 sucursales creadas
Primera sucursal de la lista: {'direccion': 'Dolorem quasi delectus', 'region': 'V Region', 'tiempo_offline_horas': 0}


In [8]:
# 216 costos mensuales (18 años)

year = 2000
month = 1

for _ in range(216):
    if month == 12:
        year += 1
        month = 1
        
    costos.append(get_costos(year, month))
    month += 1
    

print(len(costos), 'costos creados')
print('Primer costo de la lista:', costos[0])

216 costos creados
Primer costo de la lista: {'gastos_admin': 3114555, 'gastos_no_operacionales': 4301821, 'arriendos': 29241893, 'fecha_mensual': '2000-01-16'}


In [9]:
# 1000 productos

for _ in range(1000):
    productos.append(get_producto())

print(len(productos), 'productos creados')
print('Primer producto de la lista:', productos[0])

1000 productos creados
Primer producto de la lista: {'nombre': 'Sunt', 'categoria': 'Chandal', 'precio': 64000}


In [10]:
# 60 empleados

for _ in range(60):
    empleados.append(get_empleado())

print(len(empleados), 'empleados creados')
print('Primer empleado de la lista:', empleados[0])

60 empleados creados
Primer empleado de la lista: {'nombre': 'Elvia Cortez Irizarry', 'cargo': 'Cajero', 'sueldo': 561941, 'fecha_ingreso': '2011-04-19', 'fecha_desvinculacion': '2016-02-20', 'horas_trabajadas': '26262'}


In [11]:
# 30000 ratings

for _ in range(3000):
    ratings.append(get_rating())

print(len(ratings), 'ratings creados')
print('Primer rating de la lista:', ratings[0])

3000 ratings creados
Primer rating de la lista: {'cant_estrellas': 4, 'comentarios': 'Deleniti quod eos quam id iure cumque. Nisi fugiat illum reprehenderit. Beatae ratione quae asperiores neque.'}


In [12]:
# 500.000 clientes

for _ in range(500000):
    clientes.append(get_cliente())

print(len(clientes), 'clientes creados')
print('Primer cliente de la lista:', clientes[0])

500000 clientes creados
Primer cliente de la lista: {'fecha_creacion': '2005-03-06', 'fecha_ultima_compra': '2005-08-11', 'fecha_ultima_devolucion': '2001-06-25', 'total_compras': 84, 'cantidad_devoluciones': 20}


## Guardar Datos en DataFrame y Exportarlos a CSV

In [13]:
# sucursales

data = [list(x.to_dict().values()) for x in sucursales]
cols = sucursales[0].to_dict().keys()
df = pd.DataFrame(data, columns=cols)
df.to_csv('../datasets/sucursales.csv')

In [14]:
# empleados

data = [list(x.to_dict().values()) for x in empleados]
cols = empleados[0].to_dict().keys()
df = pd.DataFrame(data, columns=cols)
df.to_csv('../datasets/empleados.csv')

In [15]:
# costos mensuales

data = [list(x.to_dict().values()) for x in costos]
cols = costos[0].to_dict().keys()
df = pd.DataFrame(data, columns=cols)
df.to_csv('../datasets/costos_mensuales.csv')

In [16]:
# productos

data = [list(x.to_dict().values()) for x in productos]
cols = productos[0].to_dict().keys()
df = pd.DataFrame(data, columns=cols)
df.to_csv('../datasets/productos.csv')

In [17]:
# ratings

data = [list(x.to_dict().values()) for x in ratings]
cols = ratings[0].to_dict().keys()
df = pd.DataFrame(data, columns=cols)
df.to_csv('../datasets/ratings.csv')

In [18]:
# clientes

data = [list(x.to_dict().values()) for x in clientes]
cols = clientes[0].to_dict().keys()
df = pd.DataFrame(data, columns=cols)
df.to_csv('../datasets/clientes.csv')